# Analysis of demo_rom_manager.py - Narrowing Example

### demo_rom_manager.py

1. Gets the parameters for the ROM Manager

`general_rom_manager_parameters = GetRomManagerParameters()`

`project_parameters_name = "ProjectParameters.json"`

2. Creates a RomManager object using the parameters from the ProjectParameters.json file, the rom manager pararameters from gereral_rom_manager_parameters, a customizer function for the simulation and a customizer function for project parameters

`rom_manager = RomManager(project_parameters_name,general_rom_manager_parameters,CustomizeSimulation,UpdateProjectParameters)`

3. Runs the `Fit()` method

4. Calls the `PrintErros` method and finishes the execution

### RomManager Object

Besides the four parameters defined when creating the object, it also has default `rom_training_parameters`

#### Rom Training Parameters:

Gets default values from _GetDefaulRomBasisOutputParameters

Sets `rom_manager` to 'True'

Updates the default values using the provided `general_rom_parameters`, if necessary

### RomManager Fit

**Why is mu None? Which parameters could be added here? And how?**

`mu_train` is defined as a simple string if not defined earlier

The project strategy is executed, according to what is defined at the parameters. Considering Galerkin for this analysis.

The training_stages (in this case, 'ROM') are loaded into the `training_stages` variable.

The `fom_snapshots` is created using the `LaunchTrainROM` method

The simulation parameters are changed to GalerkinROM using the `_ChangeRomFlags` method

The `rom_snapshots` is created using the `LaunchROM` method

`ROMvsFOM_train` is calculated using the norm of rom and fom snapshots **The snapshots are really similar, does this mean that the reduction was very efficient?**


#### LaunchTrainRom

1. The parameters are loaded as a Parameters object, using the file defined earlier;
2. An empty snapshots matrix is created;
3. Iterates through every element in `mu_train`, keeping track of its Id

    3.1 Updates the parameters using the provided `UpdateProjectParameters` function. In this case, this does nothing.
    
    3.2 Updates the parameters using the `_AddBasisCreationToProjectParameters` method. This adds the `rom_training_parameters` parameters to `rom_output`
    
    3.3 Calls the `_StoreResultsByName` method with 'FOM' as parameter, which updates the parameters, setting the name of the output file based on the provided `results_name` and the current `Id` or `mu`
    
    3.4 Creates a Kratos Model object
    
    3.5 Creates `analysis_stage_class`, using the `_GetAnalysisStageClass` method and using the update `parameters` object. In this case, <class 'KratosMultiphysics.FluidDynamicsApplication.fluid_dynamics_analysis.FluidDynamicsAnalysis'>
    
    3.6 Creates a CustomSimulation object, using the analysis stage, model, and updated parameters
    
    3.7 Runs the simulation for the FOM **(So no reduction up to this point, right?)** , and creates the FOM0.post.bin file at the Results folder
    
    3.8 Iterates through the output processes and adds the instance of `CalculateRomBasisOutputProcess` to the `BasisOutputProcess` variable. (More on it later)
    
    3.9 Appends to the `SnapshotsMatrix` the returned matrix of `BasisOutputProcess._GetSnapshotsMatrix`
    
    3.10 Repeats for all Id's or mu's
    
4. The `SnapshotsMatrix` has now 1 element (because there was only one Id/mu), containing a 9216 x 450 matrix. **9216 is the number of nodes * the number of unknows. How is it ordered? For example, the first three lines are the different unknowns for the first node? Or the first n_nodes lines are the first unknown for all node, than the next n_nodes are the second unknown and so on** The numpy.block method is used, now the `SnapshotsMatrix` is concatenated. **In this case, since there as only one element, the matrix becomes the element itself. What would happen if there were more elements?**
5. The concatenated `SnapshotsMatrix` is passed as parameter for the `_PrintRomBasis` method from the `BasisOutputProcess` object **This is where reduction occurs, correct? Are we using POD here?**

    **Regarding the application of the quadratic manifold reduction, is this where it's supposed to happen? Replace the steps bellow with the new reduction?**

    5.1 This method initializes a dictionary with the default rom manager settings, and updates according to the used defined parameters
    
    5.2 The variable `n_nodal_unknowns` is set as the lenght of the `snapshot_variables_list`
    
    5.3 The randomized SVD is calculated using the `RandomizedSingularValueDecomposition` method, with the `snapshots_matrix` and `svd_truncation_tolerance` as parameters. (More on the method later) **Are we ignoring S, V and eSVD by using u, _, _, _ ?**
    
    5.4 The rom basis dictionary is updated with: a list of the names of the nodal unknows, the number of dofs (number of columns in u, which is the returned matrix of left singular values), the projection strategy and the rom format
    
    5.5 The data on `u` is saved as RightBasisMatrix.npy **u is the reduced order matrix, correct?**, and the list of nodes as NodeIds.npy **the number of lines in RightBasisMatrix is 3 times the number of nodes (because we have 3 nodal unknowns). it is still unclear to me how it is ordered**
    
    5.6 Saves the rom basis dictionary to RomParameters.json
    
    5.7 Finally, returns the snapshots matrix for the FOM **These are the snapshots with no reduction, correct?**
    

#### LaunchRom

1. Steps 1 to 3.2 are the same here as in LaunchTrainRom

    .
    .
    .

    3.3 Calls the `_StoreResultsByName` method with 'ROM' as parameter, which updates the parameters, setting the name of the output file based on the provided `results_name` and the current `Id` or `mu`
    
    3.4 Creates a Kratos Model object
    
    3.5 Creates `analysis_stage_class`, using the `SetUpSimulationInstance(model, parameters)` **What is the difference here to using _GetAnalysisStageClass? I see we create a simulation instance here just to get its type and then ignore it. Could we use it to get the `simulation` object as well?**
    
    3.6 Creates a CustomSimulation object, using the analysis stage, model, and updated parameters **Here couldn't we use the simulation we just generated at 3.5?**
    
    3.7 Runs the simulation for the ROM, and creates the ROM0.post.bin file at the Results folder
    
    3.8 Iterates through the output processes and adds the instance of `CalculateRomBasisOutputProcess` to the `BasisOutputProcess` variable.
    
    3.9 Appends to the `SnapshotsMatrix` the returned matrix of `BasisOutputProcess._GetSnapshotsMatrix`
    
    3.10 Repeats for all Id's or mu's
    
4. The `SnapshotsMatrix` has now 1 element (because there was only one Id/mu), containing a 9216 x 450 matrix. The numpy.block method is used, now the `SnapshotsMatrix` is concatenated.
5. Finally, returns the snapshots matrix for the ROM **Same shape as the FOM snapshots**

### CalculateRomBasisOutputProcess

Is defined as "A process to set the snapshots matrix and calculate the ROM basis from it." **Is it an instance of KratosMultiphysics.OutputProcess?**

Starts by validating input settings against the defaults. Gets model_part name, snapshots_control_type and snapshots_interval from the settings.

Gets and nodal_unknowns and sort them alphabetically. Creates a snapshot_variable_list using Kratos Global Variables.

Gets the output format (numpy in this case), rom_basis_output_name and svd_truncation_tolerance.

Initializes the output interval data at 0.0

Initializes the snapshots data list as an empty list.

Gets `rom_manager` boolean value **What is the use of this?**

#### _GetSnapShotMatrix(self):

Creates an empty numpy array of shape (number of nodal unknowns * number of nodes) x (number of data cols) **Where does n_data_cols come from? It seems like it's the number of steps. Does it come as a process result from the .Run() method?**

Creates an auxiliary list `aux_col` with the results of every node at the current step.

Saves the list, tranposed, as the i_col column of the `snapshots_matrix`. In this case, because the `aux_col` has only 1 dimension, the original and the transposed arrays are the same. 

Returns the `snapshots_matrix`


### RandomizedSingularValueDecomposition

This function initializes by defining:

1. whether to return the matrix of left singular vectors U (`COMPUTE_U`)
2. whether to return the matrix of right singular vectors V (`COMPUTE_V`)
3. how to deal with truncation tolerance (`RELATIVE_SVD`)
4. whether to use randomization or the regular numpy svd algorithm (`USE_RANDOMIZATION`)

#### Calculate(matrix, truncation_tolerance):

1. If using randomization, gets Q, B, eORTH and a from _RandomizedOrthogonalization method bellow, being:

Q: numpy array containing the orthonormal basis such that norm(A - Q@Q.T@A) <= mu
B: numpy array Q.T@A
eORTH: numpy array estimation of the orthogonalization error (nC)
a : norm of A 

2. if Q is empty, sets everything to zero or empty and returns
3. otherwise, checks if Q (result of randomized orthogonalization of A) is full rank
    3.1 if so, sets U, S, V and eSVD using _SingularValueDecompostionTruncated method and A matrix
    3.2 if not, updates the truncation tolerance and runs the _SingularValueDecompostionTruncated method with B matrix, updates the returned U with `U = Q @ U` and the eSVD with `eSVD = np.sqrt(eORTH**2 + eSVDb**2)`
    
4. returns U,S,V,eSVD

#### _RandomizedOrthogonalization(matrix):

1. Gets the dimensions of the matrix (M,N)
2. Calculates the norm of C and saves it in `c` and `nC`
3. Estimates the mu, if none is given **Where does this equation comes from? Halko et al, 2019?**
4. `dRmax` is the minimum value between M and N, divided by four and rounded up **What is the meaning of the dR's? delta R?**
5. `dRmin` is the mininum value between M, N and 1 **won´t this be always 1?**
6. `dRmin` is updated using the max of the current dRmin and; 5% of the minimum between M and N, rounded up
7. The initial guess for R is defined as 0.5% of the minimum between M and N, rounded up, and TypeRankEstimate is defined to 1 (Exponential) **Does it make sense to have a TypeRankEstimate = 1 parameter, so we can change it if necessary?**
8. Sets the initial values for dR (= R), i (= 1, the counter), nC_old (= c, temporary variable to keep track of nC) and R_old (= 0, temporary variable to keep track of R). Q and B are initialized as empty numpy arrays.
9. The function now loops continuously, untill the orthogonalization error is less or equal to the machine precision parameter (mu):
    
    9.1 Omega = a random N x dR matrix
    
    9.2 nOmega is the square root of the the shape of the matrix C (9126 x 450 at the first loop)
    
    9.3 factorRed is defined as 10 **Factor of Reduction? Why 10? Does it make sense to have a factorRed = 10 parameter, so we can change it if necessary?**
    
    9.4 sets the maximum size of SVD as the maximum between M and N, divided by factorRed
    
    9.5 calculates Qi(M x dR) using reduced qr factorization (numpy.linalg.qr) of (C @ Omega / nOmega). Throws away the Ri **correct?**
    
    9.6 if Qi is empty, breaks the loop
    
    9.7 if Q is not empty, reorthogonalizes: calculates a new SVD_MaxSize as the max(row, column) of Qi; updates Qi using another qr factorization, this time of:
    
        (Qi - Q @ (Q.T @ Qi))
        
    9.8 Computes new residual with `Bi = Qi.T @ C` and `C = C - Qi @ Bi`
    
    9.9 At the first loop, populates Q and B with Qi and Bi, respectively
    
    9.10 At the other loops, updates `Q = np.c_[Q,Qi]` and `B = np.r_[B,Bi]` **I little confused as to how np.c_ and np.r_ works. Will take a more careful look on that later**
    
    9.11 Computes the norm of residual of the updated matrix C, and prints the iteration information at the screen
    
    9.12 R_new is defined as the number of columns in matrix Q
    
    9.13 estimates the rank according to the `TypeRankEstimate` variable
    
    9.14 updates `dR` (first rounds up to the difference between `Rest` and `R_new`, then updates itself the minimum between `dR` and `dRmax`, and finally sets itself as the maximum of `dR` and `dRmin`
    
    9.15 updates the `Rest` by adding `R_new` and the updated `dR`
    
    9.16 checks whether `Rest` is equal or greater than the number of columns at C (`N`). If so, sets Q as "FULL", B as an empty array and breaks the loop
    
    9.17 updates the values of i, R_old, nC_old and R, and repeats the loop
    
    9.18 when the loop is over, returns Q, B, nC and c
    
